<a href="https://colab.research.google.com/github/buddypia/openai-agents-sdk-colab/blob/master/research_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai-agents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.2/100.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.5 MB/s eta 0:00:00


In [ ]:
import os
from google.colab import userdata

# ColabのシークレットからOpenAIのAPIキーを取得
OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")

# OpenAIエージェントにAPIキーを設定
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [ ]:
from pydantic import BaseModel

from agents import Agent

PROMPT = (
    "あなたは役に立つリサーチアシスタントです。クエリが与えられた場合、"
    "そのクエリに最もよく答えるために実行する一連のウェブ検索を考案してください。"
    "検索する用語を5から20個の間で出力してください。"
)

# WebSearchItemモデル：個々のウェブ検索アイテムを定義
class WebSearchItem(BaseModel):
    reason: str
    """この検索がクエリにとって重要である理由についてのあなたの推論。"""

    query: str
    """ウェブ検索に使用する検索語。"""


class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem]
    """クエリに最もよく答えるために実行するウェブ検索のリスト。"""

# プランナーエージェントを定義
planner_agent = Agent(
    name="PlannerAgent",
    instructions=PROMPT,
    model="gpt-4o",
    output_type=WebSearchPlan,
)

In [ ]:
from agents import Agent, WebSearchTool
from agents.model_settings import ModelSettings

INSTRUCTIONS = (
    "あなたはリサーチアシスタントです。検索語が与えられたら、その語でウェブを検索し、"
    "結果の簡潔な要約を作成します。要約は2〜3段落、300語未満でなければなりません。"
    "主要なポイントを捉えてください。簡潔に書き、完全な文や正しい文法は不要です。"
    "これはレポートを作成する人が利用するため、本質を捉え、冗長な部分を無視することが不可欠です。"
    "要約自体以外の追加のコメントは含めないでください。"
)

# サーチエージェントを定義
search_agent = Agent(
    name="Search agent",
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool()],
    model_settings=ModelSettings(tool_choice="required"),
)

In [ ]:
from pydantic import BaseModel

from agents import Agent

PROMPT = (
    "あなたは、ある調査クエリに対する一貫性のあるレポートを作成する任務を負った、シニアリサーチャーです。\n"
    "元のクエリと、リサーチアシスタントが行ったいくつかの初期調査が提供されます。\n"
    "まず、レポートの構造と流れを記述するレポートのアウトラインを作成する必要があります。\n"
    "次に、レポートを生成し、それを最終的な出力として返します。\n"
    "最終的な出力はマークダウン形式である必要があり、長く詳細なものにする必要があります。\n"
    "コンテンツは5〜10ページ、少なくとも1000語を目指してください。"
)

class ReportData(BaseModel):
    short_summary: str
    """調査結果の短い（2〜3文の）要約。"""

    markdown_report: str
    """最終レポート"""

    follow_up_questions: list[str]
    """さらなる調査のための推奨トピック"""

# ライターエージェントを定義
writer_agent = Agent(
    name="WriterAgent",
    instructions=PROMPT,
    model="o3-mini",
    output_type=ReportData,
)

In [ ]:
import asyncio
import time

from agents import Runner, custom_span, gen_trace_id, trace

class ResearchManager:
  async def run(self, query: str) -> None:
      # 検索計画を立てる
      search_plan = await self._plan_searches(query)
      # 検索を実行する
      search_results = await self._perform_searches(search_plan)
      # レポートを作成する
      report = await self._write_report(query, search_results)

      # 最終レポート（短い要約）
      final_report = f"レポート要約\n\n{report.short_summary}"

      print("\n\n=====レポート=====\n\n")
      print(f"レポート: {report.markdown_report}")
      print("\n\n=====追加質問=====\n\n")
      follow_up_questions = "\n".join(report.follow_up_questions)
      print(f"追加質問: {follow_up_questions}")

  async def _plan_searches(self, query: str) -> WebSearchPlan:
    # プランナーエージェントを実行して検索計画を取得
    result = await Runner.run(
      planner_agent,
      f"クエリ: {query}", # ユーザーの元のクエリ
    )

    # 結果をWebSearchPlanオブジェクトとして返す
    return result.final_output_as(WebSearchPlan)

  async def _perform_searches(self, search_plan: WebSearchPlan) -> list[str]:
      # "ウェブ検索" という名前でカスタムスパンを開始
      with custom_span("ウェブ検索"):
          num_completed = 0 # 完了した検索タスクの数
          # 検索計画内の各項目に対して非同期検索タスクを作成
          tasks = [asyncio.create_task(self._search(item)) for item in search_plan.searches]
          results = [] # 検索結果を格納するリスト
          # 完了したタスクから順に結果を処理
          for task in asyncio.as_completed(tasks):
              result = await task # タスクの結果を取得
              if result is not None: # 結果がNoneでない場合のみ追加
                  results.append(result)
              num_completed += 1 # 完了数をインクリメント
          return results # 検索結果のリストを返す

  async def _search(self, item: WebSearchItem) -> str | None:
      # 検索エージェントへの入力を作成
      input = f"検索語: {item.query}\n検索理由: {item.reason}"
      try:
          # 検索エージェントを実行
          result = await Runner.run(
              search_agent,
              input,
          )
          # 結果を文字列として返す
          return str(result.final_output)
      except Exception:
          # エラーが発生した場合はNoneを返す
          return None

  async def _write_report(self, query: str, search_results: list[str]) -> ReportData:
      # ライターエージェントへの入力を作成
      input = f"元のクエリ: {query}\n要約された検索結果: {search_results}"
      # ライターエージェントをストリームモードで実行
      result = await Runner.run(
          writer_agent,
          input,
      )

      # 最終的な結果をReportDataオブジェクトとして返す
      return result.final_output_as(ReportData)

In [ ]:
query = "サイバーエージェントのAI事業本部の業務効率化のポイントについて"

In [ ]:
await ResearchManager().run(query)

ERROR:openai.agents:No active trace. Make sure to start a trace with `trace()` firstReturning NoOpSpan.




=====レポート=====


レポート: # サイバーエージェントのAI事業本部における業務効率化のポイントに関する詳細レポート

本レポートでは、サイバーエージェントのAI事業本部がどのようにして業務効率化を推進しているのか、そしてその背後にある技術、組織戦略、課題、そして未来への展望について検証していきます。レポートは以下のセクションに分かれており、各セクションでは関連する取り組み、事例、技術背景について詳細に解説します。

---

## 1. はじめに

サイバーエージェントは1998年の設立以来、デジタル広告、メディア、エンターテインメント分野で革新的なビジネスモデルを展開してきました。その中でも、2019年に発足したAI事業本部は、広告配信技術の革新や新規事業の創出を目的に設立され、AI技術とデジタルトランスフォーメーション（DX）を強力に推進しています。このレポートでは、業務効率化の観点から、同事業本部がどのようなツールや戦略を採用しているのか、またその取り組みがどのようにして企業全体の生産性向上に寄与しているのかを明らかにします。


## 2. サイバーエージェントのAI事業本部の概要

### 2.1 組織構造と役割

サイバーエージェントのAI事業本部は、以下の主要な組織で構成されています：

- **AI本部:** 高度なAIプロダクトの開発と提供を担い、広告配信の予測モデルやクリエイティブ制作におけるAI技術を推進。
- **DX本部:** 小売、医療、デジタルガバメントなど、さまざまな業界におけるデジタル変革（DX）を支援。
- **D2C本部:** 自社ブランドの創出と運営を担当し、AI技術を活用した市場分析および販売戦略を展開。
- **Data Science Center:** データサイエンスの専門チームとして、各サービスの品質・精度向上を実現。
- **AI Lab:** 機械学習、自然言語処理、コンピュータビジョンなど、多様なAI分野の研究開発の最前線として、大学や研究機関との連携を推進。

### 2.2 歴史と背景

2019年のAI事業本部の設立背景には、デジタル広告業界におけるAI技術の重要性の高まりと、新たなビジネスモデルの創出が求められたことがあります。近年、生成AIの登場により、社内外の業務効率化の可能